In [ ]:
import math
import numpy as np
import pandas as pd

# IR wavelength 2711.16 nm, 6 ns pulse width, 7.2 mJ/pulse
# Path length, L (sets optimal concentration)

# Now, relative humidity of 100% means that 4.19% (maximum) of the molecules are water vapor for a given volume of atmosphere.
# So, RH = 50% means 80*0.0419% = 3.352% of molecules are water vapor
# If n_h2o and n_total are the number density of water molecules and all molecules, respectively in per cubic meter

WL = 2711.16 # beam wavelength in nm
SL = 299792458 # speed of light in m/s
P = 101325 # 1 atm = 101325 pa
R = 8.314 # Universal gas constant
RH = 50 # Relative humidity
TC = 20 # ambient temp in °C
T = 273 + TC # ambient temp in Kelvin

data = [
    ['T_C', 'T_F', 'P_kPa', 'P_torr', 'P_atm'],
      [0, 32, 0.6113, 4.5851, 0.006],
      [5, 41, 0.8726, 6.545, 0.0086],
      [10, 50, 1.2281, 9.2115, 0.0121],
      [15, 59, 1.7056, 12.7931, 0.0168],
      [20, 68, 2.3388, 17.5424, 0.0231],
      [25, 77, 3.169, 23.7695, 0.0313],
      [30, 86, 4.2455, 31.8439, 0.0419],
      [35, 95, 5.6267, 42.2037, 0.0555],
      [40, 104, 7.3814, 55.3651, 0.0728],
      [45, 113, 9.5898, 71.9294, 0.0946],
      [50, 122, 12.344, 92.5876, 0.1218],
      [55, 131, 15.752, 118.1497, 0.1555],
      [60, 140, 19.932, 149.5023, 0.1967],
      [65, 149, 25.022, 187.6804, 0.2469],
      [70, 158, 31.176, 233.8392, 0.3077],
      [75, 167, 38.563, 289.2463, 0.3806],
      [80, 176, 47.373, 355.3267, 0.4675],
      [85, 185, 57.815, 433.6482, 0.5706],
      [90, 194, 70.117, 525.9208, 0.692],
      [95, 203, 84.529, 634.0196, 0.8342],
      [100, 212, 101.32, 759.9625, 1],
    ]
# source of 'data' dataset: https://en.wikipedia.org/wiki/Vapour_pressure_of_water

col_P_atm = 5 # column number in the 'data' dataset
df = pd.DataFrame(data)
ind = np.where(df == TC)
rowInd, colInd = ind[0], ind[1]
val = df.iloc[rowInd, col_P_atm-1]
p_sat_T = val.iloc[0]

print ('Vapour pressure of water is', p_sat_T , 'atm at', TC,'°C temperature and', RH,'% relative humidity')

In [ ]:
p_total = 1 # total pressure is 1 atm
x_sat_h2o = RH*p_sat_T/(100*p_total)

n_total = P/(R*T)
n_h2o = x_sat_h2o*n_total #in mol/m3 unit
n_h2o_cm = n_h2o*(pow(10, -6)) # in mol/cm3 unit
NA = 6.023*pow(10, 23) # Avogadro's number
n_h2o_N = n_h2o*NA
n_h2o_N_cm = n_h2o_N*(pow(10, -6)) # in mol/cm3 unit

print("Water vapor in mol per cubic centimeter", "{:.4e}".format(n_h2o_cm))
# print("Water vapor in mol per cubic meter", "{:.4f}".format(n_h2o))
print("Number of water vapor molecule in per cubic centimeter ","{:.4e}".format(n_h2o_N_cm))
# print("Number of water vapor molecule in per cubic meter ","{:.4e}".format(n_h2o_N))

xsf = 5 # Excited-state cross-section factors from page 10 of Pitz 2000 (Unseeded molecular flow tagging in cold and hot flows using ozone and hydroxyl tagging velocimetry)

σ_i = xsf*8*pow(10, -22)*pow(10, -4) # absorption cross-section = 5×8×10^-22 cm^2 (our target case; 5 times of ground state according to Pitz 2000)
E = 1*pow(10, -3) # laser pulse energy imposed on water vapors = 1 mJ (assumed that 5 mJ weakens to 1 mJ)
h = 6.62607015*pow(10, -34) # Planck’s constant in J⋅s unit
nu = SL/WL # frequency of the light at 193 nm
# L = 12*0.3048 # length of the H2O laden region through which the laser beam travels in meter (1 ft = 0.3048 m)
A = math.pi*pow(0.7/2, 2)*pow(10, -6) # area of the laser beam when beam dia is 0.7 mm

ndi = n_h2o*σ_i*E/(h*nu*A)
ndi_cm = ndi*(pow(10, -6)) # in mol per cubic cm
ndi_N =  n_h2o_N*σ_i*E/(h*nu*A)
ndi_N_cm = ndi_N*(pow(10, -6)) # in mol per cubic cm

# print("Water vapor dissociated in mol per cubic meter", "{:.4e}".format(ndi))
print("Water vapor dissociated in mol per cubic centimeter", "{:.4e}".format(ndi_cm))
# print("Number of water vapor molecule dissociated in per cubic meter ","{:.4e}".format(ndi_N))
print("Number of water vapor molecule dissociated in per cubic centimeter ","{:.4e}".format(ndi_N_cm))

In [ ]:
E = 3.8183 # Molar absorption coefficient, E = 3.8183 M-1cm-1 at WL = 2711.16 nm from HITRAN
L = 82 # IR lser optical path length, L = 82 cm
C = n_h2o_cm*1000 # Water vapor concentration, C
I0_I_ratio = math.pow(10,E*L*C)
print('Ratio of incident light intencity and transmitted light intencity at water Vapour pressure of', p_sat_T , 'atm, temperature of', TC,'°C  and relative humidity of', RH,'% is',I0_I_ratio)
print('So, loss due to absorption is', "{:.2f}".format((1-1/I0_I_ratio)*100),'%')